In [2]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams.update({"font.size": 12})
%matplotlib inline

In [4]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#from spacytextblob.spacytextblob import SpacyTextBlob
import json

In [1]:
!pip3 install spacymoji==2.0.0
!pip3 uninstall spacy -y
!pip3 install spacy==3

     |████████████████████████████████| 51 kB 2.9 MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49735 sha256=fee8516a9468ee48a992bacac25bed2c50ead88e34059c8772acb60e911a95f7
  Stored in directory: /root/.cache/pip/wheels/4e/bf/6b/2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji
Found existing installation: spacy 2.2.4
Uninstalling spacy-2.2.4:
  Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 12.7 MB 3.7 MB/s 
     |████████████████████████████████| 628 kB 41.6 MB/s 
     |████████████████████████████████| 9.1 MB 27.1 MB/s 
     |████████████████████████████████| 42 kB 664 kB/s 
     |████████████████████████████████| 451 kB 49.0 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv("/content/drive/MyDrive/stocks/stocks_submissions_all.csv", sep=";")
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
df.index = pd.DatetimeIndex(df.created_utc)
df.tail()

,Unnamed: 0,author,created_utc,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio
created_utc,,,,,,,,,,,
2018-12-30 02:45:51,154501,ChowSu,2018-12-30 02:45:51,aarxpu,NaN,13,1,306865,NaN,Stocks and taxes?,0.0
2018-12-30 02:29:05,154502,analyst_84,2018-12-30 02:29:05,aart2j,NaN,25,1,306864,NaN,Best of 2018 returns,0.0
2018-12-30 01:57:05,154503,01Cloud01,2018-12-30 01:57:05,aark2v,NaN,9,1,306855,NaN,Best bear ETFs,0.0
2018-12-30 01:32:01,154504,SilverInitiatives,2018-12-30 01:32:01,aarcpu,NaN,34,1,306853,NaN,Theoretical: Shouldn't stock prices be infinity?,0.0
2018-12-30 00:26:31,154505,19392o3o3u2,2018-12-30 00:26:31,aaqssi,NaN,4,1,306831,NaN,I need advice from a profesional,0.0


# Split datasets 

In [7]:
df['year'] = pd.DatetimeIndex(df['created_utc']).year

In [8]:
df['month'] = pd.DatetimeIndex(df['created_utc']).month
df['date'] = pd.DatetimeIndex(df['created_utc']).date
df['day'] = pd.DatetimeIndex(df['created_utc']).day
df.head()

,Unnamed: 0,author,created_utc,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio,year,month,date,day
created_utc,,,,,,,,,,,,,,,
2021-11-26 13:19:32,0,NovelAnteater,2021-11-26 13:19:32,r2nb9a,NaN,0,1,3310630,[removed],SNAP seems like a great long-term buy at this ...,1.0,2021,11,2021-11-26,26
2021-11-26 12:37:50,1,VonDerBerg,2021-11-26 12:37:50,r2ma8u,Trades,0,1,3310546,So the markets are tanking on new variant news...,$NVAX new variant,1.0,2021,11,2021-11-26,26
2021-11-26 12:36:24,2,VonDerBerg,2021-11-26 12:36:24,r2m96j,Trades,0,1,3310543,[removed],$NVAX new variant,1.0,2021,11,2021-11-26,26
2021-11-26 12:29:36,3,VonDerBerg,2021-11-26 12:29:36,r2m3di,Trades,0,1,3310525,[removed],$NVAX lift-off confirmed,1.0,2021,11,2021-11-26,26
2021-11-26 12:10:01,4,ghdsvor,2021-11-26 12:10:01,r2lrdw,Advice Request,0,1,3310479,[removed],Where to find dates that annual reports were p...,1.0,2021,11,2021-11-26,26


In [9]:
df['link_flair_text'] = df['link_flair_text'].astype("string")

In [10]:
df.head(3)

,Unnamed: 0,author,created_utc,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio,year,month,date,day
created_utc,,,,,,,,,,,,,,,
2021-11-26 13:19:32,0,NovelAnteater,2021-11-26 13:19:32,r2nb9a,<NA>,0,1,3310630,[removed],SNAP seems like a great long-term buy at this ...,1.0,2021,11,2021-11-26,26
2021-11-26 12:37:50,1,VonDerBerg,2021-11-26 12:37:50,r2ma8u,Trades,0,1,3310546,So the markets are tanking on new variant news...,$NVAX new variant,1.0,2021,11,2021-11-26,26
2021-11-26 12:36:24,2,VonDerBerg,2021-11-26 12:36:24,r2m96j,Trades,0,1,3310543,[removed],$NVAX new variant,1.0,2021,11,2021-11-26,26


## group by user

In [11]:
df_authors = df.groupby("author")["author"].count().sort_values(ascending=False)
print(f"Top 15 authors:\n{df_authors.head(15)}")

Top 15 authors:
author
[deleted]               3871
coolcomfort123          1016
AutoModerator            914
bigbear0083              330
psychotrader00           226
luchins                  181
gorays21                 179
apooroldinvestor         149
InterestingNews1         137
0toHeroInvesting         128
NXTmine                  121
rammerman123             118
Brothanogood             107
Ok-Length1319            100
QuantalyticsResearch      90
Name: author, dtype: int64


In [12]:
# remove submissions where author = '[deleted]'

In [13]:
df_deleted = df[df['author'] == '[deleted]']
df = df[df['author'] != '[deleted]']

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 150635 entries, 2021-11-26 13:19:32 to 2018-12-30 00:26:31
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Unnamed: 0             150635 non-null  int64         
 1   author                 150635 non-null  object        
 2   created_utc            150635 non-null  datetime64[ns]
 3   id                     150635 non-null  object        
 4   link_flair_text        65867 non-null   string        
 5   num_comments           150635 non-null  int64         
 6   score                  150635 non-null  int64         
 7   subreddit_subscribers  150635 non-null  int64         
 8   selftext               131974 non-null  object        
 9   title                  150635 non-null  object        
 10  upvote_ratio           150635 non-null  float64       
 11  year                   150635 non-null  int64         
 12  month     

In [15]:
df_deleted.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3871 entries, 2021-11-24 09:05:21 to 2019-01-06 20:54:22
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             3871 non-null   int64         
 1   author                 3871 non-null   object        
 2   created_utc            3871 non-null   datetime64[ns]
 3   id                     3871 non-null   object        
 4   link_flair_text        2080 non-null   string        
 5   num_comments           3871 non-null   int64         
 6   score                  3871 non-null   int64         
 7   subreddit_subscribers  3871 non-null   int64         
 8   selftext               667 non-null    object        
 9   title                  3871 non-null   object        
 10  upvote_ratio           3871 non-null   float64       
 11  year                   3871 non-null   int64         
 12  month                  387

In [16]:
df_authors = df.groupby("author")["author"].count().sort_values(ascending=False)
print(f"Top 15 authors:\n{df_authors.head(20)}")

Top 15 authors:
author
coolcomfort123          1016
AutoModerator            914
bigbear0083              330
psychotrader00           226
luchins                  181
gorays21                 179
apooroldinvestor         149
InterestingNews1         137
0toHeroInvesting         128
NXTmine                  121
rammerman123             118
Brothanogood             107
Ok-Length1319            100
No_Seesaw1134             90
QuantalyticsResearch      90
theepicone111             90
futureIsYes               88
BurnerBurnerBurns20       87
juaggo_                   83
coronastocks              81
Name: author, dtype: int64


In [17]:
df_daily_title= df.groupby(['author']).agg({'title': ' '.join})
#df_daily_title= df.groupby(pd.Grouper(freq="D")).agg({'title': ' '.join})
#df.groupby(['name','month'])['text'].apply(','.join).reset_index()

In [18]:
df_author_posts= df.groupby(['author']).agg({'id': 'count'})

In [19]:
df_author_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84608 entries, ----The_Truth----- to zzzzoooo
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      84608 non-null  int64
dtypes: int64(1)
memory usage: 1.3+ MB


In [20]:
df_author_posts.head()

,id
author,
----The_Truth-----,1
---Lancelot---,1
---thisismyusername,1
--Arts--,1
--Redi-Jedi--,2


In [21]:
df_author_posts['title']= df.groupby(['author']).agg({'title': ' '.join})

In [22]:
df_author_posts.head()

,id,title
author,,
----The_Truth-----,1,How to play ETFs
---Lancelot---,1,I don’t want to sound dumb but
---thisismyusername,1,CTL is getting killed after hours for cutting ...
--Arts--,1,"UiPath, Inc. (PATH) buy?"
--Redi-Jedi--,2,"AKBA filed an NDA, is it a good time to jump o..."


In [23]:
df['selftext'] = df['selftext'].astype("string")

In [24]:
df.selftext = np.where(df.selftext.isnull()," ",df.selftext)

In [25]:
df_author_posts['selftext'] = df.groupby(['author']).agg({'selftext': ' '.join})

In [26]:
df['date'] = df['date'].astype("string")

In [27]:
df_author_posts['date']= df.groupby(['author']).agg({'date': ' '.join})

In [28]:
df['day'] = df['day'].astype("string")
df['month'] = df['month'].astype("string")
df['year'] = df['year'].astype("string")

In [29]:
df_author_posts['day']= df.groupby(['author']).agg({'day': ' '.join})
df_author_posts['month']= df.groupby(['author']).agg({'month': ' '.join})
df_author_posts['year']= df.groupby(['author']).agg({'year': ' '.join})

In [30]:
df_author_posts.head()

,id,title,selftext,date,day,month,year
author,,,,,,,
----The_Truth-----,1,How to play ETFs,I've been trading pennies and swinging &lt;$50...,2020-10-06,6,10,2020
---Lancelot---,1,I don’t want to sound dumb but,When is the best time to get in for GME specif...,2021-01-23,23,1,2021
---thisismyusername,1,CTL is getting killed after hours for cutting ...,,2019-02-14,14,2,2019
--Arts--,1,"UiPath, Inc. (PATH) buy?",[removed],2021-06-10,10,6,2021
--Redi-Jedi--,2,"AKBA filed an NDA, is it a good time to jump o...",[removed] [removed],2021-03-30 2021-03-30,30 30,3 3,2021 2021


## number of users who have posted more than n times

In [31]:
df_author_posts[df_author_posts['id'] > 10].info()

<class 'pandas.core.frame.DataFrame'>
Index: 985 entries, -0-itisaplane to zeroskater45
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        985 non-null    int64 
 1   title     985 non-null    object
 2   selftext  985 non-null    object
 3   date      985 non-null    object
 4   day       985 non-null    object
 5   month     985 non-null    object
 6   year      985 non-null    object
dtypes: int64(1), object(6)
memory usage: 61.6+ KB


In [32]:
df_author_posts[df_author_posts['id'] > 1].info()

<class 'pandas.core.frame.DataFrame'>
Index: 23238 entries, --Redi-Jedi-- to zzzzoooo
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        23238 non-null  int64 
 1   title     23238 non-null  object
 2   selftext  23238 non-null  object
 3   date      23238 non-null  object
 4   day       23238 non-null  object
 5   month     23238 non-null  object
 6   year      23238 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.4+ MB


In [33]:
df_author_posts[(df_author_posts['id'] > 10) & (df_author_posts['id'] <50)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 939 entries, -0-itisaplane to zeroskater45
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        939 non-null    int64 
 1   title     939 non-null    object
 2   selftext  939 non-null    object
 3   date      939 non-null    object
 4   day       939 non-null    object
 5   month     939 non-null    object
 6   year      939 non-null    object
dtypes: int64(1), object(6)
memory usage: 58.7+ KB


In [34]:
df_author_posts[df_author_posts['id'] > 50].info()

<class 'pandas.core.frame.DataFrame'>
Index: 43 entries, 0toHeroInvesting to theepicone111
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        43 non-null     int64 
 1   title     43 non-null     object
 2   selftext  43 non-null     object
 3   date      43 non-null     object
 4   day       43 non-null     object
 5   month     43 non-null     object
 6   year      43 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.7+ KB


In [35]:
df_author_posts[df_author_posts['id'] > 100].info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 0toHeroInvesting to rammerman123
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        12 non-null     int64 
 1   title     12 non-null     object
 2   selftext  12 non-null     object
 3   date      12 non-null     object
 4   day       12 non-null     object
 5   month     12 non-null     object
 6   year      12 non-null     object
dtypes: int64(1), object(6)
memory usage: 768.0+ bytes


#Tickers by user

In [36]:

nasdaq_stocks = pd.read_csv("/content/drive/MyDrive/WSB/nasdaq.csv", sep=",")
nasdaq_stocks_symbols = nasdaq_stocks["Symbol"].tolist()

In [37]:
# Function to find stocks in a text given stock list
import re
regex = re.compile("[^a-zA-Z ]")
ticker_set = set(nasdaq_stocks_symbols)
#list(filter(lambda word: word[0]=='s', text.split()))

def calculate_mentioned_stocks(title, body=""):
    #content = regex.sub("", str(title) + " " + str(body)).split(" ")
    text = str(title) + " " + str(body)   
    tick_list = list(filter(lambda word: word[0]=='$', text.split())) 
    content = set(list([i[1:] for i in tick_list]))
    tickers = str(list(ticker_set & content))    
    return pd.Series([tickers])

def sanitize_column(column):
    #print(str(column))
    column = str(column).replace("[],", "").replace("[]", "").replace("],[", ",").replace("][", ",")
    return column

In [38]:
df_author_posts['tickers_'] = df_author_posts.apply(lambda x: calculate_mentioned_stocks(x["title"],x["selftext"]), axis=1)


In [40]:
df_author_posts['tickers_'] = df_author_posts['tickers_'].apply(sanitize_column)

In [41]:
df_author_posts.head()

,id,title,selftext,date,day,month,year,tickers_
author,,,,,,,,
----The_Truth-----,1,How to play ETFs,I've been trading pennies and swinging &lt;$50...,2020-10-06,6,10,2020,
---Lancelot---,1,I don’t want to sound dumb but,When is the best time to get in for GME specif...,2021-01-23,23,1,2021,
---thisismyusername,1,CTL is getting killed after hours for cutting ...,,2019-02-14,14,2,2019,
--Arts--,1,"UiPath, Inc. (PATH) buy?",[removed],2021-06-10,10,6,2021,
--Redi-Jedi--,2,"AKBA filed an NDA, is it a good time to jump o...",[removed] [removed],2021-03-30 2021-03-30,30 30,3 3,2021 2021,


In [42]:
df_author_posts['tickers_'].tolist()[:10]

['', '', '', '', '', '', '', '', '', '']

In [43]:
df_author_posts['tickers']= df_author_posts['tickers_'].apply(lambda x: sanitize_column(x))

In [44]:
df_author_posts['tickers'].tolist()[:10]

['', '', '', '', '', '', '', '', '', '']

In [45]:
from collections import Counter, defaultdict
import ast

In [46]:
#count stocks
df_author_posts["tickers"] =df_author_posts["tickers"].str.strip("[]").str.split(",")
df_author_posts["tickers_count"] = df_author_posts.apply(lambda x: dict(Counter(list(x['tickers']))), axis=1)

In [47]:
from numpy.core.numeric import NaN

def agg_counter_objs(rows):
    #print(rows)
    #print(type(rows))
    if ((rows == '') | (rows == NaN)):
        return 0
    new_counter_objs = []
    #rows= ast.literal_eval(rows)
    def default_val():
        return 0
        
    new_dict = defaultdict(default_val)
    #print(rows)
    test_dict = Counter(rows)
    for key in test_dict.keys():
        new_key = key.replace("'","").replace('"',"").replace("]","").replace("\\","").strip()
        new_dict[new_key] += test_dict[key]
            
    if "" in new_dict.keys():
        del new_dict[""]
            
    new_counter_objs.append(Counter(new_dict))
        
    agg_counter_obj = Counter({})
    for obj in new_counter_objs:
        agg_counter_obj += obj        
    return (str(dict(agg_counter_obj)))

In [48]:
df_author_posts['tickers_count_'] = df_author_posts["tickers_count"].apply(agg_counter_objs)

In [50]:
df_author_posts = df_author_posts.drop(['tickers','tickers_count'],axis=1)

In [51]:
df_author_posts.head()

,id,title,selftext,date,day,month,year,tickers_,tickers_count_
author,,,,,,,,,
----The_Truth-----,1,How to play ETFs,I've been trading pennies and swinging &lt;$50...,2020-10-06,6,10,2020,,{}
---Lancelot---,1,I don’t want to sound dumb but,When is the best time to get in for GME specif...,2021-01-23,23,1,2021,,{}
---thisismyusername,1,CTL is getting killed after hours for cutting ...,,2019-02-14,14,2,2019,,{}
--Arts--,1,"UiPath, Inc. (PATH) buy?",[removed],2021-06-10,10,6,2021,,{}
--Redi-Jedi--,2,"AKBA filed an NDA, is it a good time to jump o...",[removed] [removed],2021-03-30 2021-03-30,30 30,3 3,2021 2021,,{}


In [52]:
df['link_flair_text'] = df['link_flair_text'].astype("string")

In [53]:
df['link_flair_text'] = np.where(df.link_flair_text.isnull(),"",df.link_flair_text)

In [54]:
df_author_posts['link_flair_text']= df.groupby(['author']).agg({'link_flair_text': ' '.join})

In [55]:
df_author_posts['comments_mean']= df.groupby(['author']).agg({'num_comments': 'mean'})
df_author_posts['comments_sum']= df.groupby(['author']).agg({'num_comments': 'sum'})

In [56]:
df_author_posts['score_mean']= df.groupby(['author']).agg({'score': 'mean'})
df_author_posts['score_sum']= df.groupby(['author']).agg({'score': 'sum'})

In [57]:
df_author_posts['upvote_ratio_mean']= df.groupby(['author']).agg({'upvote_ratio': 'mean'})
df_author_posts['upvote_ratio_sum']= df.groupby(['author']).agg({'upvote_ratio': 'sum'})

In [58]:
df_author_posts.head()

,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum
author,,,,,,,,,,,,,,,,
----The_Truth-----,1,How to play ETFs,I've been trading pennies and swinging &lt;$50...,2020-10-06,6,10,2020,,{},ETFs,17.0,17,1.0,1,1.0,1.0
---Lancelot---,1,I don’t want to sound dumb but,When is the best time to get in for GME specif...,2021-01-23,23,1,2021,,{},Question,7.0,7,1.0,1,1.0,1.0
---thisismyusername,1,CTL is getting killed after hours for cutting ...,,2019-02-14,14,2,2019,,{},,15.0,15,1.0,1,0.0,0.0
--Arts--,1,"UiPath, Inc. (PATH) buy?",[removed],2021-06-10,10,6,2021,,{},,0.0,0,1.0,1,1.0,1.0
--Redi-Jedi--,2,"AKBA filed an NDA, is it a good time to jump o...",[removed] [removed],2021-03-30 2021-03-30,30 30,3 3,2021 2021,,{},,0.0,0,1.0,2,1.0,2.0


## Count Flairs

In [59]:
df_author_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84608 entries, ----The_Truth----- to zzzzoooo
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 84608 non-null  int64  
 1   title              84608 non-null  object 
 2   selftext           84608 non-null  object 
 3   date               84608 non-null  object 
 4   day                84608 non-null  object 
 5   month              84608 non-null  object 
 6   year               84608 non-null  object 
 7   tickers_           84608 non-null  object 
 8   tickers_count_     84608 non-null  object 
 9   link_flair_text    84608 non-null  object 
 10  comments_mean      84608 non-null  float64
 11  comments_sum       84608 non-null  int64  
 12  score_mean         84608 non-null  float64
 13  score_sum          84608 non-null  int64  
 14  upvote_ratio_mean  84608 non-null  float64
 15  upvote_ratio_sum   84608 non-null  float64
dtypes: floa

In [60]:
df_author_posts['flairs'] = df_author_posts['link_flair_text'].apply(lambda x: list(x.split(" ")))


In [61]:
df_author_posts["flairs_count"] = df_author_posts['flairs'].apply(lambda x: dict(Counter(list(x))))

In [62]:
df_author_posts.head(3)

,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count
author,,,,,,,,,,,,,,,,,,
----The_Truth-----,1,How to play ETFs,I've been trading pennies and swinging &lt;$50...,2020-10-06,6,10,2020,,{},ETFs,17.0,17,1.0,1,1.0,1.0,[ETFs],{'ETFs': 1}
---Lancelot---,1,I don’t want to sound dumb but,When is the best time to get in for GME specif...,2021-01-23,23,1,2021,,{},Question,7.0,7,1.0,1,1.0,1.0,[Question],{'Question': 1}
---thisismyusername,1,CTL is getting killed after hours for cutting ...,,2019-02-14,14,2,2019,,{},,15.0,15,1.0,1,0.0,0.0,[],{'': 1}


# Sentiment by Authors

In [ ]:
!pip install SpacyTextBlob

In [64]:
from spacytextblob.spacytextblob import SpacyTextBlob

In [65]:
# nlp = spacy.load("en_core_web_lg")
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.blank("en")
if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob")
    nlp.rename_pipe("spacytextblob", "sentiment")
print(nlp.pipe_names)

['sentiment']


In [66]:

def get_sentiment(title, body):
    if(len(body) < 3689602):
      nlp_title = nlp(title)
      nlp_body = nlp(str(body))
      title_tokens = [token.text for token in nlp_title]
      body_tokens = [token.text for token in nlp_body]
      #ngrams = ngrams.append(list(textacy.extract.basics.ngrams(nlp_title, 3)))
      #print(pd.Series([nlp_title._.polarity, nlp_title._.subjectivity,
      #                  nlp_body._.polarity, nlp_body._.subjectivity,title_tokens,body_tokens]))
      return pd.Series([nlp_title._.polarity, nlp_title._.subjectivity,
                        nlp_body._.polarity, nlp_body._.subjectivity,title_tokens,body_tokens])
    else:
      return pd.Series([0, 0,
                       0, 0,0,0])



In [67]:
nlp.max_length = 3689602  
df_author_posts[["title_polarity", "title_subjectivity", "body_polarity", "body_subjectivity","title_tokens","selftext_tokens"]] = df_author_posts.apply(lambda row: get_sentiment(row["title"], row["selftext"]), axis=1)


In [68]:
df_author_posts["polarity"] = (df_author_posts["title_polarity"] + df_author_posts["body_polarity"])

In [69]:
df_author_posts.head()

,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity
author,,,,,,,,,,,,,,,,,,,,,,,,,
----The_Truth-----,1,How to play ETFs,I've been trading pennies and swinging &lt;$50...,2020-10-06,6,10,2020,,{},ETFs,17.0,17,1.0,1,1.0,1.0,[ETFs],{'ETFs': 1},0.000000,0.000000,0.102778,0.591667,"[How, to, play, ETFs]","[I, 've, been, trading, pennies, and, swinging...",0.102778
---Lancelot---,1,I don’t want to sound dumb but,When is the best time to get in for GME specif...,2021-01-23,23,1,2021,,{},Question,7.0,7,1.0,1,1.0,1.0,[Question],{'Question': 1},0.012500,0.450000,0.583333,0.333333,"[I, do, n’t, want, to, sound, dumb, but]","[When, is, the, best, time, to, get, in, for, ...",0.595833
---thisismyusername,1,CTL is getting killed after hours for cutting ...,,2019-02-14,14,2,2019,,{},,15.0,15,1.0,1,0.0,0.0,[],{'': 1},-0.322222,0.355556,0.000000,0.000000,"[CTL, is, getting, killed, after, hours, for, ...",[ ],-0.322222
--Arts--,1,"UiPath, Inc. (PATH) buy?",[removed],2021-06-10,10,6,2021,,{},,0.0,0,1.0,1,1.0,1.0,[],{'': 1},0.000000,0.000000,0.000000,0.000000,"[UiPath, ,, Inc., (, PATH, ), buy, ?]","[[, removed, ]]",0.000000
--Redi-Jedi--,2,"AKBA filed an NDA, is it a good time to jump o...",[removed] [removed],2021-03-30 2021-03-30,30 30,3 3,2021 2021,,{},,0.0,0,1.0,2,1.0,2.0,"[, ]",{'': 2},0.700000,0.600000,0.000000,0.000000,"[AKBA, filed, an, NDA, ,, is, it, a, good, tim...","[[, removed, ], [, removed, ]]",0.700000


In [70]:
df_author_posts.to_csv('/content/drive/MyDrive/stocks/stocks_users_featurs.csv',sep=';')

In [71]:
df_top_users =df_author_posts[df_author_posts['id']> 10]

In [72]:
df_top_users.to_csv('/content/drive/MyDrive/stocks/stocks_top_users_features.csv',sep=';')